In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def khatrirao_product(tensor1, tensor2):
    i0 = tf.constant(1)
    prod0 = tf.multiply(tensor1[0, :], tensor2)
    _, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(tensor1)[0]),
                              lambda i, m: [i+1, tf.concat([m, tf.multiply(tensor1[i, :], tensor2)], 0)],
                              loop_vars=[i0, prod0],
                              shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])
    return khprod

In [3]:
def rank3tensor_decomposition_ALS(matrix, k, alpha=0.01, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    costA = tf.reduce_sum(tf.abs(X0 - tf.matmul(A, tf.transpose(khatrirao_product(C, B)))))
    costB = tf.reduce_sum(tf.abs(X1 - tf.matmul(B, tf.transpose(khatrirao_product(C, A)))))
    costC = tf.reduce_sum(tf.abs(X2 - tf.matmul(C, tf.transpose(khatrirao_product(B, A)))))
    
    trainA = tf.train.GradientDescentOptimizer(alpha).minimize(costA)
    trainB = tf.train.GradientDescentOptimizer(alpha).minimize(costB)
    trainC = tf.train.GradientDescentOptimizer(alpha).minimize(costC)
    
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)

    for _ in range(nbiter):
        sess.run(trainA, feed_dict={X: x})
        sess.run(trainB, feed_dict={X: x})
        sess.run(trainC, feed_dict={X: x})

    return sess.run((A, B, C), feed_dict={X: x})

In [4]:
def rank3tensor_decomposition_ALS_closed(matrix, k, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    for _ in range(nbiter):
        A = tf.matmul(tf.matmul(X0, khatrirao_product(C, B)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(B), B)))
        B = tf.matmul(tf.matmul(X1, khatrirao_product(C, A)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(A), A)))
        C = tf.matmul(tf.matmul(X2, khatrirao_product(B, A)), 
                      tf.multiply(tf.matmul(tf.transpose(B), B), tf.matmul(tf.transpose(A), A)))
        
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)
    
    return sess.run((A, B, C), feed_dict={X: x})

# Test: Method 1

In [5]:
x1 = np.array([[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]], dtype=np.float32)
x2 = np.array([[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]], dtype=np.float32)
x = np.zeros((3, 4, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [6]:
A, B, C = rank3tensor_decomposition_ALS(x, k=5, nbiter=10000)

In [7]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [8]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(5))

In [9]:
mat

array([[[ 1.4840956 , 14.81542093],
        [ 4.03926674, 16.54217708],
        [ 7.51586095, 19.98188126],
        [ 8.6932199 , 20.91238475]],

       [[ 2.55345427, 15.31148604],
        [ 3.78485732, 15.52706909],
        [ 8.19020149, 19.96129751],
        [ 9.95800454, 21.04517984]],

       [[ 3.4563559 , 16.22834317],
        [ 4.95387781, 16.74170446],
        [ 7.22126658, 18.67638087],
        [10.69535763, 21.62662816]]])

In [10]:
x

array([[[ 1., 13.],
        [ 4., 16.],
        [ 7., 19.],
        [10., 22.]],

       [[ 2., 14.],
        [ 5., 17.],
        [ 8., 20.],
        [11., 23.]],

       [[ 3., 15.],
        [ 6., 18.],
        [ 9., 21.],
        [12., 24.]]])

# Method 2

In [23]:
A, B, C = rank3tensor_decomposition_ALS_closed(x, k=5, nbiter=1)

In [24]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [25]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(5))

In [26]:
mat

array([[[inf, inf],
        [inf, inf],
        [inf, inf],
        [inf, inf]],

       [[inf, inf],
        [inf, inf],
        [inf, inf],
        [inf, inf]],

       [[inf, inf],
        [inf, inf],
        [inf, inf],
        [inf, inf]]])

In [27]:
x

array([[[ 1., 13.],
        [ 4., 16.],
        [ 7., 19.],
        [10., 22.]],

       [[ 2., 14.],
        [ 5., 17.],
        [ 8., 20.],
        [11., 23.]],

       [[ 3., 15.],
        [ 6., 18.],
        [ 9., 21.],
        [12., 24.]]])

In [28]:
A, B, C

(array([[3901.4683 , -386.7866 , 1401.9554 , -863.4341 , -636.5982 ],
        [4040.54   , -392.85498, 1454.3479 , -894.6831 , -660.776  ],
        [4179.613  , -398.9236 , 1506.7405 , -925.93225, -684.954  ]],
       dtype=float32),
 array([[-5.6039290e+13, -3.8981507e+12, -1.0287092e+13,  6.6330978e+12,
         -1.2510161e+13],
        [-6.2633915e+13, -4.3382163e+12, -1.1516337e+13,  7.4340243e+12,
         -1.3974681e+13],
        [-6.9228535e+13, -4.7782811e+12, -1.2745585e+13,  8.2349512e+12,
         -1.5439199e+13],
        [-7.5823160e+13, -5.2183465e+12, -1.3974832e+13,  9.0358772e+12,
         -1.6903720e+13]], dtype=float32),
 array([[-inf,  inf, -inf, -inf,  inf],
        [-inf,  inf, -inf, -inf,  inf]], dtype=float32))